# Credits
Based on [DCGAN-tensorflow](https://github.com/carpedm20/DCGAN-tensorflow) by [@carpedm20](https://github.com/carpedm20) (Taehoon Kim) on GitHub.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
df_dim = 64
gf_dim = 64

batch_size = 64
sample_size = 64
z_dim = 100

image_shape = [64, 64, 3]

In [3]:
# Linear
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
            initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias

In [4]:
# Conv2D Layer
def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
                            initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')

        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())
        
        return conv

In [5]:
# Deconv2D Layer
def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="deconv2d"):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('w', [k_h, k_h, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))
        deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape, strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        return deconv

In [6]:
def discriminator(image, reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    
    h0 = tf.nn.relu(conv2d(image, df_dim, name='d_h0_conv'))
    h1 = tf.nn.relu(conv2d(image, df_dim * 2, name='d_h1_conv'))
    h2 = tf.nn.relu(conv2d(image, df_dim * 4, name='d_h2_conv'))
    h3 = tf.nn.relu(conv2d(image, df_dim * 8, name='d_h3_conv'))
    h4 = linear(tf.reshape(h3, [batch_size, -1]), 1, 'd_h3_lin')
    
    return tf.nn.sigmoid(h4), h4

In [7]:
def generator(z):
    h0 = tf.nn.relu(tf.reshape(linear(z, gf_dim * 8 * 4 * 4, 'g_h0_lin'), [-1, 4, 4, gf_dim * 8]))
    h1 = tf.nn.relu(deconv2d(h0, [batch_size, 8, 8, gf_dim*4], name='g_h1'))
    h2 = tf.nn.relu(deconv2d(h1, [batch_size, 16, 16, gf_dim*2], name='g_h2'))
    h3 = tf.nn.relu(deconv2d(h2, [batch_size, 32, 32, gf_dim*1], name='g_h3'))
    h4 = deconv2d(h3, [batch_size, 64, 64, 3], name='g_h4')

    return tf.nn.tanh(h4)

In [8]:
# Make the model
images = tf.placeholder(tf.float32, [batch_size] + image_shape, name='real_images')
sample_images= tf.placeholder(tf.float32, [sample_size] + image_shape, name='sample_images')
z = tf.placeholder(tf.float32, [None, z_dim], name='z')

G = generator(z)
D, D_logits = discriminator(images)
D_, D_logits_ = discriminator(G, reuse=True)

d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(D_logits, tf.ones_like(D)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(D_logits_, tf.zeros_like(D_)))
d_loss = d_loss_real + d_loss_fake

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(D_logits_, tf.ones_like(D_)))

In [9]:
# Optimizers
t_vars = tf.trainable_variables()

d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]

d_optim = tf.train.AdamOptimizer().minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer().minimize(g_loss, var_list=d_vars)